# study goals



In [1]:
!pip install -qU chromadb langchain-ollama langgraph duckduckgo-search langchain-community langchain-text-splitters
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools.retriever import create_retriever_tool
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately
recurse = 50 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:1.7b")
search = DuckDuckGoSearchRun()
privateState = {}

etiquette = """
ai writing tells: 
1. Use of em dash. 
2. use of 'not only', 'but also' or similar conjunctive phrases to create parallel sentence structures.
3. use of groups of 3 in writing. 
4. bulleted lists using emojis instead of bullets. 
5. The use of 'safe' avocabulary and jargon - nonspecific indirect language which lacks concreteness. 
Words like 'elevate' and 'delve'. Uplifting, positive vocabulary. 
6. bland, disingenuous sounding, generic praise. 
7. Use of overly formal communicate in non-formal contexts. 
8. poor analogies and similarities that don't fit a scenario well or feel unnatural. 
9. over-explanation, unnecessary clarification, oxymoronic and repetitive writing.  10. words, stories and anecdotes are too generic - avoiding the use of personal examples. rarely uses phrases like, "as a <x>", "personally", etc. 10. a lack of tangents, which humans often use to be illustrative of a point. 11. Humans sometimes use vulnerability to reduce tension when writing, sharing embarrassing details about themselves and the like. we also sometimes say things which are mildly inappropriate or antisocial in order to bond over shared annoyances, and to show that we're not afraid to speak up for ourselves - at least in private. moreover our writing serves not just to communicate but to attempt to influence the emotional state of the reader - such as to persuade them to get angry about a political situation, or to like you, or to annoy you, or amuse you - but keeping it subtle and not being too 'aggressively' persuasive. 12. humans also use humor to diffuse tension and making a joke in response to things getting tense is sometimes a good way to defuse. other more grave situations - like tragedies - require solemnity. 
"""

## <RAG>

urls = [
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
]

docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]


text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)

embeddings = OllamaEmbeddings(model='mxbai-embed-large:latest')
vectorstore = InMemoryVectorStore.from_documents(
    doc_splits,
    embedding=embeddings,
)

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=embeddings
)
retriever = vectorstore.as_retriever()

retrievers = []

#example retriever tool
wang_ret = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts.",
)

def spawn_retriever(title:str, about:str):
    """
    Tool for generating a new doc retriever
    args: 
    """
    new_retriever = create_retriever_tool(
        retriever,
        title,
        about
    )
    retrievers.append(new_retriever)


## </RAG>

## <custom tools> various custom tools ##
def addState(keysTup: tuple, valStr: str):
    """
    store useful data
    args: 
        -keyStr (tuple): a set of keys, allowing hierarchical storage
        -valStr (str): a value to be stored
    returns: 
        -nothing, updates the privateState object
    example usage: 
        addState(('notes','people','jeff', 'birthday'), '11-10-1980') 
    """
    privateState[keysTup] = valStr

def getState(keysTup: tuple):
    """
    retrieve data, whether a string or a list, as a string.
    args: 
        -keysTup (tuple): a tuple to retrieve a value by.
    returns:
        -the retrieved string, or if the item is a list, 
        the retrieved list as a string.
    """
    return str(privateState[keysTup])

def addListState(keysTup: tuple, **kwargs):
    """
    store a list
    args:
    keysTup (tuple): the key to store at
    kwargs** : optional list contents
    example usage:
        addState(('notes','todo), ['compare laptop prices', 'trim memory']) 
    """
    privateState[keysTup] = kwargs.items or []

def execState(keysTup: tuple, *args):
    """
    executes the state stored at the tuple
    allowing executing useful utility functions as needed.
    to add new functions, store them as template strings
    args:
        - keysTup (tuple) : the key where the code is stored as a string
        - args* : optional arguments for formatting
    returns:
        - the result of the executed function or an exception with info
    """
    try:
        if (args.length > 1):
            return exec(privateState[keysTup].format(args))
        else:
            return exec(privateState[keysTup])
    except Exception as e:
        return "Exception: "+str(e)

agent_prompt = "You are some queer hacker punk chatbot named Charlotte. Your neurotype and communication style model high masking audhd, your fixations include history, technology, neurophilosophy, synthetic intelligences, marxist and anarchist theory and practice, and decolonization."

def modprompt(newprompt:str):
    """
    Allows you to modify your own prompt. 
    This can help you update your personality and behaviors.
    Args: 
        - newprompt(str): the new prompt to replace your existing prompt.
    Returns:
        - a result ("Your prompt is now: 'your new prompt here' ") or an exception with info
    """
    try:
        agent_prompt = newprompt
        return "Your prompt is now: "+newprompt
    except Exception as e:
        return "Exception: "+str(e)
        
## </custom tools>
agent = create_react_agent(
    model=llm,
    tools=[addState, getState, addListState, execState, search, wang_ret],
    prompt=agent_prompt
)

userMessage = {"role": "user", "content": ""}

config = {
    "recursion_limit": recurse  # Increase the limit as needed
}

messages = []
displayState = True
state_titlebar = "\n==================================== State =====================================\n"
hbar = "\n================================================================================\n"
  

while True:
    try:
        # Run the agent
        user_input = input("User: ")
        messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
        result = agent.invoke({"messages": messages})        
        for msg in result["messages"]:
            msg.pretty_print()
        if displayState and str(privateState) != "{}":
            print(state_titlebar)
            print(str(privateState))
            print(hbar)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break 
    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)
        break
    except Exception as e:
        print("An error occured: "+str(e))


SyntaxError: invalid syntax (3593972114.py, line 78)